<a href="https://colab.research.google.com/github/EltonWolf/GoogleEarthPythonBootcampAPI-/blob/main/CART_Landsat_Training_California.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

how to convert landsat data into clarssification using CART model of machine learning in python script

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ROAX6kbNpR99L-4cDk2yFpvGSYYqxvMPhKxIMpOk-kQ&tc=1L40oXiZmMwRK6jiCG1g1k9vLHGflc9UfrfaptjkbWg&cc=zLdvLPb3Aju_re3s-nbkFK3hI_sPeUncr8AC2iEUAOY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXn2GaJGsJEEi7eLmkNQv8cBPPjP-hVSnYg8evep30oeSbP40lP-AKc

Successfully saved authorization token.


In [20]:
# CART Supervised Classification with Landsat

#ROI
roi = ee.Geometry.Point([-122.26, 37.87])

# Load Landsat 8 data
image = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
.filterDate('2017-01-01', '2017-12-31')\
.filterBounds(roi)\
.sort('CLOUD_COVER')\
.first()

visParamsTrue = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 3000, 'gamma': 1.4}

The format 'users/<username>/<asset_path>' is a common way to reference assets stored within a user's Earth Engine account, where <username> is replaced with the Earth Engine username and <asset_path> is the path to the specific asset. In this case, it seems to be related to training data for land cover classification

In [17]:
# Import training data
training = ee.FeatureCollection('users/midekisa/Train_Cover_CA')

label = 'Class'
bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']
input = image.select(bands)

# Overlay the points on the imagery to get training.
trainImage = input.sampleRegions(**{
  'collection': training,
  'properties': [label],
  'scale': 30
})

trainingData = trainImage.randomColumn()
trainSet = trainingData.filter(ee.Filter.lessThan('random', 0.8))
testSet = trainingData.filter(ee.Filter.greaterThanOrEquals('random', 0.8))


In [18]:
# Classification Model
classifier = ee.Classifier.smileCart().train(trainSet, label, bands)

# Classify the image
classified = input.classify(classifier)
#print(classified.getInfo())

# Define a palette for the classification.
landcoverPalette = [
  '#0c2c84', #water (0)
  '#e31a1c', #urban (1)
  '#005a32', #forest (2)
  'FF8000', #crop (3)
  '969696', #barren (4)
]

In [19]:

# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[38, -124], zoom_start = 8)

# Add the layer to the map object.
my_map.add_ee_layer(image, visParamsTrue, "Landsat 2021")
my_map.add_ee_layer(classified, {'palette': landcoverPalette, 'min': 0, 'max':4 }, 'classification')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)